# Gaming Tax Prediction Model - Clean Implementation

**Objective:** Build category-specific models to predict normalized gaming tax

**Key Insight:** Monthly aggregation ensures lag features have **consistent meaning** across all operators:
- lag1 = previous month (not "previous submission" which varies by operator)
- lag2 = 2 months ago  
- lag3 = 3 months ago

**Approach:**
1. Normalize Gaming Tax by Total Sales (operator-agnostic)
2. Filter by category
3. Aggregate to monthly level per operator
4. Create AR(3) lag features (each operator's past 3 months)
5. Build operator-agnostic model using only lag values

## Phase 1: Load and Normalize Data

## Phase 1: Load and Normalize Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# Load data
df = pd.read_excel('Returns to NLGRB.xlsx')
print(f"Total rows: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Clean numeric columns
df['Gaming Tax'] = pd.to_numeric(df['Gaming Tax'], errors='coerce')
df['Total Sales'] = pd.to_numeric(df['Total Sales'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Remove rows with missing critical values
df = df.dropna(subset=['Gaming Tax', 'Total Sales', 'Date', 'Operator Category', 'Operator Name'])

print(f"Rows after cleaning: {len(df)}")
print(f"\nOperator Categories: {df['Operator Category'].nunique()}")
print(df['Operator Category'].value_counts())

In [ ]:
# Normalize Gaming Tax by Total Sales (operator-agnostic metric)
df['Tax_Normalized'] = df['Gaming Tax'] / df['Total Sales']

print(f"Tax Normalized Statistics:")
print(df['Tax_Normalized'].describe())

## Phase 2: Aggregate to Monthly Level

**Why monthly?** Ensures regular intervals so lag features have consistent meaning:
- Different operators submit on different dates
- Monthly aggregation: one value per operator per month
- lag1 always means "1 month ago" for every operator

In [ ]:
# Select a category to work with
category = 'GENERAL BETTING'

# Filter data for this category
category_df = df[df['Operator Category'] == category].copy()

print(f"Category: {category}")
print(f"Total submissions: {len(category_df)}")
print(f"Unique operators: {category_df['Operator Name'].nunique()}")
print(f"Date range: {category_df['Date'].min()} to {category_df['Date'].max()}")

In [ ]:
# Create Year-Month identifier
category_df['YearMonth'] = category_df['Date'].dt.to_period('M')

print(f"\nUnique months: {category_df['YearMonth'].nunique()}")
print(f"Months: {sorted(category_df['YearMonth'].unique())}")

In [ ]:
# Aggregate to monthly level per operator
# Creates one row per operator per month

category_monthly = category_df.groupby(['Operator Name', 'YearMonth']).agg({
    'Gaming Tax': 'mean',
    'Total Sales': 'mean',
    'Total Payouts': 'mean',
    'Tax_Normalized': 'mean',
    'Date': 'first'
}).reset_index()

category_monthly = category_monthly.sort_values(['Operator Name', 'YearMonth'])

print(f"\nAfter monthly aggregation:")
print(f"Rows: {len(category_monthly)}")
print(f"Average months per operator: {len(category_monthly) / category_df['Operator Name'].nunique():.1f}")

category_monthly.head(10)

In [ ]:
# Visualize monthly time series for a sample operator
sample_operator = category_monthly['Operator Name'].value_counts().index[0]

sample_data = category_monthly[category_monthly['Operator Name'] == sample_operator]

plt.figure(figsize=(12, 4))
plt.plot(range(len(sample_data)), sample_data['Tax_Normalized'].values, marker='o')
plt.title(f'Monthly Normalized Tax: {sample_operator} ({category})')
plt.xlabel('Month Index')
plt.ylabel('Tax Normalized')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nOperator: {sample_operator}")
print(f"Number of months: {len(sample_data)}")
print(f"\nTax Normalized stats:")
print(sample_data['Tax_Normalized'].describe())

## Phase 3: Create AR(3) Lag Features

**Key concept:** 
- Lag features are created **per operator** (each operator's own history)
- But model uses only the lag values, not operator identity
- This makes it operator-agnostic while respecting temporal patterns

In [ ]:
# Create lag features: previous 3 months for each operator
category_monthly['tax_norm_lag1'] = category_monthly.groupby('Operator Name')['Tax_Normalized'].shift(1)
category_monthly['tax_norm_lag2'] = category_monthly.groupby('Operator Name')['Tax_Normalized'].shift(2)
category_monthly['tax_norm_lag3'] = category_monthly.groupby('Operator Name')['Tax_Normalized'].shift(3)

# Drop rows without all 3 lags (first 3 months per operator)
ar_data = category_monthly.dropna(subset=['tax_norm_lag1', 'tax_norm_lag2', 'tax_norm_lag3'])

print(f"Data with complete lags: {len(ar_data)} rows")
print(f"Dropped: {len(category_monthly) - len(ar_data)} rows (first 3 months per operator)")

ar_data.head(10)

In [ ]:
# Check the lag structure for sample operator
sample_op_data = ar_data[ar_data['Operator Name'] == sample_operator].head()
print(f"Lag features for {sample_operator}:")
print(sample_op_data[['YearMonth', 'Tax_Normalized', 'tax_norm_lag1', 'tax_norm_lag2', 'tax_norm_lag3']].to_string())

## Phase 4: Build Operator-Agnostic Model

**Model features:** Only lag values (lag1, lag2, lag3)  
**Target:** Tax_Normalized  
**Training:** Across all operators (learns general patterns)  
**Prediction:** Based on any operator's recent 3 months

In [ ]:
# Prepare features and target
X = ar_data[['tax_norm_lag1', 'tax_norm_lag2', 'tax_norm_lag3']]
y = ar_data['Tax_Normalized']

print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures (lag values only, no operator identity):")
print(X.head())

In [ ]:
# Train-test split (stratified by operator to ensure each operator in both sets)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42,
    stratify=ar_data['Operator Name']
)

print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

In [ ]:
# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

# Evaluate
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\nModel Performance ({category}):")
print(f"="*50)
print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Test MAE: {test_mae:.6f}")
print(f"Test RMSE: {test_rmse:.6f}")

In [ ]:
# Visualize predictions vs actual
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Tax Normalized')
plt.ylabel('Predicted Tax Normalized')
plt.title(f'Predictions vs Actual ({category})')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

✅ **Monthly aggregation** ensures regular time intervals  
✅ **Lag features per operator** capture each operator's history  
✅ **Operator-agnostic model** learns patterns across all operators  
✅ **Clean time series** with consistent monthly periods  

**Next Steps:**
- Phase 5: Detect anomalies using Modified Z-scores
- Phase 6: Repeat for all categories
- Phase 7: Generate predictions for full dataset and export to parquet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
# Load data
df = pd.read_excel('Returns to NLGRB.xlsx')
print(f"Total rows: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Clean numeric columns
df['Gaming Tax'] = pd.to_numeric(df['Gaming Tax'], errors='coerce')
df['Total Sales'] = pd.to_numeric(df['Total Sales'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Remove rows with missing critical values
df = df.dropna(subset=['Gaming Tax', 'Total Sales', 'Date', 'Operator Category', 'Operator Name'])

print(f"Rows after cleaning: {len(df)}")
print(f"\nOperator Categories: {df['Operator Category'].nunique()}")
print(df['Operator Category'].value_counts())

In [ ]:
# Normalize Gaming Tax by Total Sales (operator-agnostic metric)
df['Tax_Normalized'] = df['Gaming Tax'] / df['Total Sales']

print(f"Tax Normalized Statistics:")
print(df['Tax_Normalized'].describe())

## Phase 2: Filter by Category, then Create Time Series

In [ ]:
# Select a category to work with
category = 'GENERAL BETTING'

# Filter data for this category
category_df = df[df['Operator Category'] == category].copy()

print(f"Category: {category}")
print(f"Total submissions: {len(category_df)}")
print(f"Unique operators: {category_df['Operator Name'].nunique()}")
print(f"Date range: {category_df['Date'].min()} to {category_df['Date'].max()}")

In [ ]:
# Now average same-day values WITHIN this category
# Group by: Operator Name, Date (category already filtered)

category_daily = category_df.groupby(['Operator Name', 'Date']).agg({
    'Gaming Tax': 'mean',
    'Total Sales': 'mean',
    'Total Payouts': 'mean',
    'Tax_Normalized': 'mean'
}).reset_index()

# Sort by operator and date
category_daily = category_daily.sort_values(['Operator Name', 'Date'])

print(f"\nAfter same-day averaging:")
print(f"Rows: {len(category_daily)}")
print(f"Reduction: {len(category_df) - len(category_daily)} same-day duplicates removed")

category_daily.head(10)

In [ ]:
# Visualize time series for a sample operator
sample_operator = category_daily['Operator Name'].value_counts().index[0]

sample_data = category_daily[category_daily['Operator Name'] == sample_operator]

plt.figure(figsize=(12, 4))
plt.plot(sample_data['Date'], sample_data['Tax_Normalized'], marker='o')
plt.title(f'Normalized Tax Time Series: {sample_operator} ({category})')
plt.xlabel('Date')
plt.ylabel('Tax Normalized')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nOperator: {sample_operator}")
print(f"Number of submissions: {len(sample_data)}")
print(f"\nTax Normalized stats:")
print(sample_data['Tax_Normalized'].describe())

## Summary of Phase 1-2

✅ **Loaded and cleaned data**  
✅ **Normalized Gaming Tax** = Gaming Tax / Total Sales (operator-agnostic)  
✅ **Filtered by category** (GENERAL BETTING)  
✅ **Created time series** indexed by date  
✅ **Averaged same-day values** within the category  

**Next Steps:**
- Phase 3: Create lag features for AR model
- Phase 4: Build category-specific model
- Phase 5: Generate predictions and detect anomalies
- Phase 6: Repeat for all categories and export to parquet